# Neural Networks Implementation From Scratch
### Getting Started
The very first step would be to create a virtual enviornment and install all the requirements. You can do so by running these commands:

```bash
virtualenv venv
source venv/bin/activate
pip install -r requirements.txt
```

Then you can come back to this notebook and run the codes. 

(More information on the projects github's page [here](https://github.com/Alireza2317/NeuralNet))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Callable


In [ ]:
class NeuralNetwork:
	"""
	This class is specificly desgined for mnist handwritten digits recognition, with 784 neurons in
	the input layer, and 10 neurons in the output layer, representing each one of the digits
	the network contains 2 hidden layers, each consisting of 16 neurons.
	it can also handle different activations such as sigmoid, ReLU and tanh.
	"""
	def __init__(self, parameters: np.ndarray = np.zeros((13002, 1)), activation: str = 'sigmoid') -> None:
		if parameters.shape != (13002, 1):
			raise ValueError('parameters shape should be (13002, 1)!')
		
		self.parameters = parameters

		count = 0
		
		# weights1 is of size (16)x(28*28)
		total_size = 16*28*28
		self.weights1 = parameters[count:count+total_size].reshape(16, 28*28)

		count += total_size 

		# weights2 is of size 16x16
		total_size = 16*16
		self.weights2 = parameters[count:count+total_size].reshape(16, 16)

		count += total_size 

		# weights3 is of size 10*16
		total_size = 10*16
		self.weights3 = parameters[count:count+total_size].reshape(10, 16)

		count += total_size 

		# biases1 is of size 16x1
		total_size = 16*1
		self.biases1 = parameters[count:count+total_size].reshape(16, 1)

		count += total_size 

		# biases2 is of size 16x1
		total_size = 16*1
		self.biases2 = parameters[count:count+total_size].reshape(16, 1)
		
		count += total_size 

		# biases3 is of size 10*1
		total_size = 10*1
		self.biases3 = parameters[count:count+total_size].reshape(10, 1)

		# type of activation
		match activation:
			case 'sigmoid':
				self.activation: Callable = NeuralNetwork.sigmoid
				self.d_activation: Callable = NeuralNetwork.d_sigmoid
			case 'relu':
				self.activation: Callable = NeuralNetwork.ReLU
				self.d_activation: Callable = NeuralNetwork.d_ReLU
			case 'tanh':
				self.activation: Callable = NeuralNetwork.tanh
				self.d_activation: Callable = NeuralNetwork.d_tanh
			case 'no-activation':
				self.activation: Callable = lambda x: x
				self.d_activation: Callable = lambda x: x

		self.input_layer = np.zeros((784, 1))
		
		# inner hidden layers of the network
		self.z_hlayer1 = np.zeros((16, 1))
		self.hlayer1 = self.activation(self.z_hlayer1)

		self.z_hlayer2 = np.zeros((16, 1))
		self.hlayer2 = self.activation(self.z_hlayer2)

		self.z_output_layer = np.zeros((10, 1))
		self.output_layer = self.activation(self.z_output_layer)

		del total_size
		del count


	
	def load_input_layer(self, input_vector: np.ndarray) -> None:
		"""
		Laad the input handwritten digit
		input_vector: np.ndarray of shape (784, 1)
		these numbers are between 0-255
		the function will squish them between 0-1
		"""
		self.input_layer = (input_vector / 255)
		

	def cost_of_single_sample(self, sample: np.ndarray, true_label: int) -> float:
		self.load_input_layer(input_vector=sample)
		self.feed_forward()
		
		# construct the output vector based on the label
		desired_output = np.zeros((10, 1))
		desired_output[true_label] = 1.0
		
		# compare the self.output_layer and the desired_output
		# using mean squared error
		cost = np.sum((self.output_layer - desired_output)**2)
		return cost
	

	def cost_of_test_data(self, test_samples: np.ndarray, true_labels: np.ndarray) -> float:
		"""
		samples.shape = (784, m)
		true_labels.shape = (1, m)
		samples: is a np array which each col represents one sample, and 
		each col has 784 numbers in them, the pixel values
			
		"""

		MSE: float = 0
		for sample, label in zip(test_samples.T, true_labels[0]):
			MSE += self.cost_of_single_sample(sample, label)
		
		M = len(test_samples.T)
		MSE = (1 / M) * MSE
		return MSE
	

	def accuracy_score(self, test_samples: np.ndarray, true_labels: np.ndarray) -> float:
		#* test_samples.shape = (784, m)
		#* true_labels.shape = (1, m)

		total: int = len(test_samples.T)
		trues: int = 0

		for sample, label in zip(test_samples.T, true_labels[0]):
			result = self.predict(sample)
			if result == label:
				trues += 1
		
		return (trues / total)


	def predict_v(self, sample: np.ndarray) -> np.ndarray:
		#* sample.shape = (784, 1)
		self.load_input_layer(input_vector=sample)
		self.feed_forward()
		
		return self.output_layer

	
	def predict(self, sample) -> int:
		output_vector = self.predict_v(sample)
		return np.argmax(output_vector)


	def backprop_one_sample(self, sample: np.ndarray, label: int) -> tuple:
		"""
		This method holds all the math and calculus behind backpropagation
		it calculates the derivitive of the cost w.r.t all the weights and
		biases of the network, for only one training data
		"""
		
		self.load_input_layer(input_vector=sample)
		self.feed_forward()

		#* convert the label in int format into a one-hot vector
		desired_output = np.zeros(self.output_layer.shape)
		desired_output[label] = 1.0
		
		#* d_cost_p_ol.shape = (10, 1)
		d_cost_p_ol = 2 * (self.output_layer - desired_output)

		#* d_activation(z_ol)
		#* times the gradient of the cost w.r.t activations of the output layer
		#* error_ol.shape = (10, 1)
		error_ol = self.d_activation(self.z_output_layer) *  d_cost_p_ol

		#* d_activation(self.z_hlayer2).shape = (16, 1)
		#* self.weights3.T.shape = (10, 16).T -> (16, 10)
		#* error_ol.shape = (10, 1)
		#* error_hl2.shape = (16, 1)
		error_hl2 = self.d_activation(self.z_hlayer2) * (self.weights3.T @ error_ol)


		#* d_activation(self.z_hlayer1).shape = (16, 1)
		#* self.weights2.T.shape = (16, 16).T -> (16, 16)
		#* error_hl2.shape = (16, 1)
		#* error_hl1.shape = (16, 1)
		error_hl1 = self.d_activation(self.z_hlayer1) * (self.weights2.T @ error_hl2)
		
		#* d_cost_p_b3.shape = output_layer.shape = (10, 1)
		d_cost_p_b3 = error_ol
	

		#* d_cost_p_b2.shape = hlayer2.shape = (16, 1)
		d_cost_p_b2 = error_hl2


		#* d_cost_p_b1.shape = hlayer1.shape = (16, 1)
		d_cost_p_b1 = error_hl1

		#* the derivative of the cost wr to the weights of the layer l will be
		#* the matrix mult of error of layer l and activation of layer l-1 transposed
		
		#* error_ol.shape = (10, 1)
		#* self.hlayer2.T.shape = (16, 1).T = (1, 16)
		#* d_cost_p_w3.shape = (10, 16)
		d_cost_p_w3 = (error_ol @ self.hlayer2.T)
		

		#* error_hl2.shape = (16, 1)
		#* self.hlayer1.T.shape = (16, 1).T = (1, 16)
		#* d_cost_p_w2.shape = (16, 16)
		d_cost_p_w2 = (error_hl2 @ self.hlayer1.T)
		
		
		#* error_hl1.shape = (16, 1)
		#* self.input_layer.T.shape = (784, 1).T = (1, 784)
		#* d_cost_p_w1.shape = (16, 784)
		d_cost_p_w1 = (error_hl1 @ self.input_layer.T)


		return (
			d_cost_p_w1,
			d_cost_p_b1,
			d_cost_p_w2,
			d_cost_p_b2,
			d_cost_p_w3,
			d_cost_p_b3
		)

	
	def backpropagation(self, x_train: np.ndarray, y_train: np.ndarray) -> tuple:
		"""
		This method will run the backprop_one_sample method for a dataset and 
		take the average of all the gradients of the weights and biases
		"""

		#* m training samples
		#* x_train.shape = (784, m)
		#* y_train.shape = (1, m)
		
		
		dw1 = np.zeros(self.weights1.shape)
		db1 = np.zeros(self.biases1.shape)
		
		dw2 = np.zeros(self.weights2.shape)
		db2 = np.zeros(self.biases2.shape)
		
		dw3 = np.zeros(self.weights3.shape)
		db3 = np.zeros(self.biases3.shape)

		
		for features, label in zip(x_train.T, y_train[0]):
			#* label: int
			#* features.shape = (784, 1)
			
				
			#* label in this method should be an int
			(
				tdw1, 
				tdb1, 
				tdw2, 
				tdb2, 
				tdw3, 
				tdb3
			) = self.backprop_one_sample(sample=features, label=label)
			
			dw1 += tdw1
			db1 += tdb1
			dw2 += tdw2
			db2 += tdb2
			dw3 += tdw3
			db3 += tdb3

		#* now the dw1, db1, dw2, db2, dw3, db3 contain the sum of the derivatives of 
		#* the samples inside the training data
		#* now they should be divided by the number of the train sample size, so dw and db, be an average
		train_data_size = x_train.shape[1]
		
		dw1 /= train_data_size
		db1 /= train_data_size
		dw2 /= train_data_size
		db2 /= train_data_size
		dw3 /= train_data_size
		db3 /= train_data_size
		
		#* now they contain the gradient of the provided dataset
		return (
			dw1,
			db1,
			dw2,
			db2,
			dw3,
			db3
		)
			

	def update_parameters(self, dw1, db1, dw2, db2, dw3, db3, learning_rate) -> None:
		self.weights1 -= learning_rate * dw1
		self.biases1 -= learning_rate * db1
		
		self.weights2 -= learning_rate * dw2
		self.biases2 -= learning_rate * db2

		self.weights3 -= learning_rate * dw3
		self.biases3 -= learning_rate * db3


	def train(
			self, 
			x_train: np.ndarray, 
			y_train: np.ndarray,
			*,
			learning_rate: float = 0.1,
			constant_lr: bool = False,
			number_of_epochs: int = 80,
			mini_batches_size: int = 100
	) -> None:
		"""Trains the model with the labeled training data"""
		
		#* m training samples
		#* x_train.shape = (784, m)
		#* y_train.shape = (1, m)
		
		#* first we'd better attach the x_train and y_train together
		#* then we can shuffle the training data
		#* and then seperate the x and y again
		#* add y row to the 0-th row of train_data
		train_data = np.vstack((y_train, x_train))

		#! now because shuffle, shuffles the array based on the rows
		#! but we need to shuffle the data based on the coloumns
		#! we have to transpose it twice to get around this
		train_data = train_data.T
		np.random.shuffle(train_data)
		train_data = train_data.T

		#* now that the data is shuffled properly
		#* we should divide the data into mini-batches
		mini_batches: list[np.ndarray] = []
		count: int = 0
		while True:
			if count >= len(train_data.T):
				break
			
			mini_batches.append(train_data[:, count:count+mini_batches_size])
			count += mini_batches_size

		#* keep track of the accuracy scores, to plot later	
		scores: list[float] = []

		initial_lr: float = learning_rate
		decay_rate: float = 0.1
		for epoch in range(number_of_epochs):
			#* now each mini-batch corresponds to one step at gradient descent
			#* batch.shape = (784+1, mini_batches_size), the label and the features
			for batch in mini_batches:
				#* batch.shape = (785, mini_batches_size)
				x_train_batch = batch[1:] # (784, mini_batches_size)
				y_train_batch = batch[0:1] # (1, mini_batches_size)
				
				#* the backprop algorithm will run for each batch, one step downhill towards a local minima
				#* it also updates the self.output_layer and consequently the cost by running self.feed_forward
				dw1, db1, dw2, db2, dw3, db3 = self.backpropagation(x_train_batch, y_train_batch)

				if constant_lr:
					lr = learning_rate
				else:
					lr = np.exp(-epoch * decay_rate) * initial_lr


				#* change each of the weights and biases accordingly
				self.update_parameters(dw1, db1, dw2, db2, dw3, db3, learning_rate=lr)
			score = self.accuracy_score(x_train, y_train)*100				
			scores.append(score)
			print(f'epoch {epoch+1}: {score:.2f}%')

		self.plot_scores(list(range(1, number_of_epochs+1)), scores)

		
	def plot_scores(self, epochs_range: list[int], scores: list[float]) -> None:
		fig = plt.figure()
		fig.tight_layout()
		plt.title('Perfecting Accuracy Score')
		plt.xlabel('Epoch')
		plt.ylabel('Accuracy Score')
		plt.xticks(epochs_range)
		plt.plot(epochs_range, scores)
		plt.show()
				
	@staticmethod
	def sigmoid(z: np.ndarray) -> np.ndarray:
		return 1 / (1 + np.exp(-z))

	@staticmethod
	def d_sigmoid(z: np.ndarray) -> np.ndarray:
		return np.exp(-z) / (np.pow((1 + np.exp(-z)), 2))


	@staticmethod
	def ReLU(z: np.ndarray) -> np.ndarray:
		return np.maximum(0, z)
	
	
	@staticmethod
	def d_ReLU(z: np.ndarray) -> np.ndarray:
		return (z > 0).astype(np.float64)
	

	@staticmethod
	def tanh(z: np.ndarray) -> np.ndarray:
		return np.tanh(z)

	
	@staticmethod
	def d_tanh(z: np.ndarray) -> np.ndarray:
		return 4 * np.exp(2 * z) / np.power(np.exp(2*z) + 1, 2)


	@staticmethod
	def softmax(z: np.ndarray) -> np.ndarray:
		return np.exp(-z) / np.sum(np.exp(-z))


	def feed_forward(self) -> None:
		"""
		Will calculate all the values in all the layers 
		based on the weights and biases 
		"""
		# hidden layer 1 has 16 neurons -> shape: (16, 1)
		# weights1.shape = (16, 784)
		# 16*784	784*1 + 16,1
		self.input_layer = self.input_layer.reshape((784, 1))

		self.z_hlayer1 = self.weights1 @ self.input_layer + self.biases1
		self.hlayer1 = self.activation(self.z_hlayer1)

		# hidden layer 2 has 16 neurons -> shape: (16, 1)
		self.z_hlayer2 = self.weights2 @ self.hlayer1 + self.biases2
		self.hlayer2 = self.activation(self.z_hlayer2)
		
		# output layer has 10 neurons -> shape(10, 1)
		# one neuron for each digit
		self.z_output_layer = self.weights3 @ self.hlayer2 + self.biases3
		self.output_layer = self.activation(self.z_output_layer)


		if self.output_layer.shape != (10, 1):
			raise ValueError(f'{self.output_layer.shape} not a correct shape!')


	def print_network(self, hidden_layers = False) -> None:
		if hidden_layers:
			print('hLayer 1:')
			print(self.hlayer1)
			
			print('hLayer 2:')
			print(self.hlayer2)

		print('Output Layer:')
		print(self.output_layer)


	def plot_network(self):
		layers = [
			self.hlayer1.reshape((16,)),
			self.hlayer2.reshape((16,)),
			self.output_layer.reshape((10,))		
		]

		fig, ax = plt.subplots()
		
		# Determine the positions for the layers
		layer_sizes = [len(layer) for layer in layers]
		v_spacing = max(layer_sizes) + 1 + 3  # Vertical spacing between neurons
		h_spacing = 15  # Horizontal spacing between layers
		
		# Iterate over layers
		for i, layer in enumerate(layers):
			y_pos = np.linspace(v_spacing / 2, -v_spacing / 2, len(layer))
			x_pos = np.full_like(y_pos, i * h_spacing)
			
			# Plot neurons
			for j, neuron_value in enumerate(layer):
				circle = plt.Circle((x_pos[j], y_pos[j]), radius=0.4, facecolor=str(neuron_value), edgecolor='k')
				ax.add_artist(circle)
				# Optional: Add text to show the neuron value
				ax.text(x_pos[j]+1.5, y_pos[j], f'{neuron_value:.2f}', ha='center', va='center', color='black')

			# Draw connections (optional)
			if i > 0:
				prev_layer = layers[i - 1]
				prev_y_pos = np.linspace(v_spacing / 2, -v_spacing / 2, len(prev_layer))
				prev_x_pos = np.full_like(prev_y_pos, (i - 1) * h_spacing)
				for k in range(len(prev_layer)):
					for l in range(len(layer)):
						ax.plot([prev_x_pos[k], x_pos[l]], [prev_y_pos[k], y_pos[l]], 'k-', lw=0.04)

		ax.set_aspect('equal')
		ax.axis('off')
		plt.tight_layout()
		plt.show()


	def show_sample(self, sample: np.ndarray):
		#* sample.shape = (784, 1)
		matrix = np.array(sample).reshape(28, 28)

		plt.imshow(matrix, cmap='gray', vmin=0, vmax=255)
		plt.axis('off')
		plt.show()

	
	def print_stat(self, x_test: np.ndarray, y_test: np.ndarray) -> None:
		score = self.accuracy_score(x_test, y_test)
		cost = self.cost_of_test_data(x_test, y_test)

		print(f'Accuracy = {score * 100:.2f}%')
		print(f'Cost = {cost:.3f}')
	

### Data preparation
First of all you need to prepare the dataset in csv format. 
You can visit [this link](https://www.kaggle.com/datasets/hojjatk/mnist-dataset) on kaggle and download the zip file.

Then you need to extract the zip file, and place these four files in the `data` directory. (as the same level as the `generate_mnist_csv.py` file)
- t10k-images.idx3-ubyte
- t10k-labels.idx1-ubyte
- train-images.idx3-ubyte
- train-labels.idx1-ubyte

Now when you run `generate_mnist_csv.py` file, two csv files should be created.


In [35]:
!python3 ./data/generate_mnist_csv.py

hello


In [ ]:
train_df = pd.read_csv('./data/mnist_train.csv')
test_df = pd.read_csv('./data/mnist_test.csv')

x_train = np.array(train_df.iloc[:, 1:]).transpose() # shape = 784 * m, so each col is a sample
y_train = np.array(train_df.iloc[:, 0:1]).reshape((1, -1)) # shape = 1 * m

x_test = np.array(test_df.iloc[:, 1:]).transpose()
y_test = np.array(test_df.iloc[:, 0:1]).reshape((1, -1))

In [ ]:
#* initialize the parameters randomly
ps = np.random.normal(size=(13002, 1)) / 2
		
NN = NeuralNetwork(parameters=ps, activation='sigmoid')

In [ ]:
NN.print_stat(x_test, y_test)

In [ ]:
NN.train(x_train, y_train, number_of_epochs=5, mini_batches_size=150, learning_rate=1, constant_lr=False)

In [ ]:
NN.print_stat(x_train, y_train)
NN.print_stat(x_test, y_test)

In [ ]:
custom_df = pd.read_csv('./data/self_handwritten_digits/custom_data.csv')
custom_test_x = np.array(custom_df).T

for i, sample in enumerate(custom_test_x.T):
	pred = NN.predict(sample)
	print(f'true = {i}')
	print(f'prediction = {pred}')
	if i == pred:
		print('CORRECT!')
	else:
		print('WRONG!')
	
	NN.show_sample(sample)


In [ ]:
#* examples where the prediction was not correct
count = 0
for i, (sample, label) in enumerate(zip(x_test.T, y_test[0])):
	p = NN.predict(sample)
	if p != label:
		NN.show_sample(sample)
		print(f'true label: {label}')
		print(f'prediction: {p}')
		count += 1
		if count == 10:
			break

layers = [NN.hlayer1.reshape((16,)), NN.hlayer2.reshape((16,)), NN.output_layer.reshape((10,))]